In [1]:
import bnn
import os
import cv2
import json
from PIL import Image
from IPython.display import display
from pynq import Xlnk
import pynq
import numpy
import yaml 
import math
import time
import requests
import matplotlib.pyplot as plt
import threading
from datetime import datetime
import _thread


def yaml_dump(file_path, data):
    with open(file_path, "a") as file_descr:
        yaml.dump(data, file_descr)


def yaml_dump_write(file_path, data):
    with open(file_path, "w") as file_descr:
        yaml.dump(data, file_descr)
        
        
parking_id="60c37a011c5c3f1648817ddb"
area="A8"
area_url = "http://bkparking.ddns.net:3002/parkinglots/"+ parking_id + "/area/slot/"
coordinate_url = area_url + "coordinate/" + area
coordinates_file = "/home/xilinx/jupyter_notebooks/bnn/coordinates/c5_night.yaml"

if(os.path.exists(coordinates_file)):
    files = open(coordinates_file)
    data=yaml.load(files)
else: 
    data = requests.request("GET", url=coordinate_url).json()
    yaml_dump_write(coordinates_file, data)

print(data)

thresh_hold = 0
coords = '/home/xilinx/jupyter_notebooks/bnn/coordinates/'
video_path = '/home/xilinx/jupyter_notebooks/bnn/video/'
#Instantiate the classifier:
hw_classifier = bnn.CnvClassifier(bnn.NETWORK_CNVW2A2,'cifar10',bnn.RUNTIME_HW)
#Read coordiantes from file
# file = open(os.path.join(coords, "c5_night.yaml"))
# data = yaml.load(file)
startX, startY =0 ,0
video = cv2.VideoCapture(os.path.join(video_path,"thesis.mp4"))
# video = cv2.imread("./people/frame1.jpg")
arr_size = len(data) 
result = [0] * arr_size
send_result = [0] * arr_size
mini_count = 0
count = 0
send_interval = 3
timer = 0
send_flag = 0
s_count = 0
latency = 0
            
def send_data(send_arr):
    global send_flag, mini_count, s_count, start, latency
#     while True:
    s_count += 1
#         send_flag == False
#         print(send_result)
    sending_time = time.time()
    obj = {
                    "edge_id": "Demo Edge",
          "time":{
            "sent": str(sending_time)
          },  
          "area": {
            "name": "A8",
            "slots": send_arr
          }
    }
    send_status = requests.request("PUT", url = area_url, json = obj).status_code
#     latency += time.time() - sending_time
#     print(obj)#, sending_time)#, latency
    print("{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}\t{:.5f}".format(send_arr[0], send_arr[1], send_arr[2], send_arr[3], send_arr[4]))
#     print(str(s_count) + " " + str(send_status))
#         if send_status == 200:
#             print('Send success')
#             break

                        
        
# update_status = threading.Thread(target=send_data, args=(), daemon=True)
# update_status.start()
if __name__ == '__main__':
    lcount = 0
    fps = 0
    
#     global count,mini_count,result, send_flag
    t  = time.time()
    global send_flag, start
    print("Start")
    start_a = time.time()
#     start = time.time()
    while True:
        start = time.time()
        count += 1
        mini_count += 1
        images = []
#         img = cv2.imread('./people/frame1.jpg')
        success, img = video.read()
        if success == False:
            break
        #Extract each slot for classify-----------
#         img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        for line in data:
            startX, startY, width, height = line['point_array']
            crop = img[startY:startY + height, startX:startX + width]
            crop = cv2.resize(crop, (32,32))
            images.append(crop)
#         s = time.time()
        rankingW2A2 = hw_classifier.classify_images_details(images)
#         e = time.time()
#         fps += (1/(e-s))
#         print("FPS: {:.3f}".format( 1/(e-s)))    
        for i in range(0, len(images)):
#             if rankingW2A2[i*len(hw_classifier.classes)+1] >= thresh_hold and rankingW2A2[i*len(hw_classifier.classes)] < 0:
            if rankingW2A2[i*len(hw_classifier.classes)+1] > rankingW2A2[i*len(hw_classifier.classes)+0]:
                result[i] += 1
#         print("Done...")
        #     Show ranking of each class 
#         for i in range(0, len(images)):
#             display(Image.fromarray(images[i]))
#             print("\n{: >10}{: >13}".format("[CLASS]","[RANKING]"))
#             for j in range s(0, len(hw_classifier.classes)):
#                 print("{: >10}{: >10}".format(hw_classifier.classes[j],rankingW2A2[i*len(hw_classifier.classes)+j]))
#             print("\n")
        timer += time.time() - start
#         print(timer)
        if timer >= send_interval:
            lcount += 1
            global send_flag
            send_flag = True
            send_result = [slot/mini_count for slot in result]
            _thread.start_new_thread(send_data,(send_result,))
            result = [0] * arr_size
            mini_count = 0
            timer = 0
        end = time.time()
#         print("FPS: {:.9f}".format( 1/(end-start))) 
        fps += 1/(end-start)
#         if count == 1000:
#             break
#     print("Total time: "+str(time.time() - start_a))
#     print("Average FPS: {:.3f}".format(fps/count)) 
#     print(latency/count)
#     print(lcount)
#     global s_count
#     print(s_count)

#     print("\n^^DONE^^\n")     

[{'id': '1', 'point_array': [424, 5, 165, 88]}, {'id': '2', 'point_array': [471, 54, 214, 103]}, {'id': '3', 'point_array': [578, 116, 258, 144]}, {'id': '4', 'point_array': [706, 195, 341, 232]}, {'id': '5', 'point_array': [896, 328, 381, 387]}]
Start
0.00000	0.02703	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.01282	0.00000	0.00000	0.00000
0.01266	0.06329	0.00000	0.00000	0.00000
0.00000	0.14474	0.00000	0.92105	0.00000
0.00000	0.21053	0.00000	1.00000	0.00000
0.00000	0.12821	0.00000	1.00000	0.00000
0.00000	0.10390	0.00000	1.00000	0.00000
0.00000	0.05263	0.00000	1.00000	0.00000
0.00000	0.00000	0.00000	0.89744	0.00000
0.00000	0.00000	0.66234	0.06494	0.00000
0.55844	0.70130	0.06494	0.00000	0.00000
0.41558	0.28571	0.00000	0.00000	0.00000
0.00000	0.11538	0.00000	0.00000	0.00000
